In [ ]:
%pip install --force-reinstall --no-cache-dir tensorflow
%pip install --force-reinstall --no-cache-dir keras

In [4]:
%pip install -U tensorflow

  Using cached ml_dtypes-0.3.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
Using cached ml_dtypes-0.3.2-cp311-cp311-macosx_10_9_universal2.whl (389 kB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
Note: you may need to restart the kernel to use updated packages.


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
#from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [15]:
# Load the dataset
df = pd.read_csv("input/spamorham.csv")

# Preprocessing the data

In [16]:
print(df.shape)
df.dropna(inplace=True)
print(df.shape)

(20348, 2)
(20348, 2)


In [17]:
df.head()

,text_type,text
0,spam,naturally irresistible your corporate identity...
1,spam,the stock trading gunslinger fanny is merrill ...
2,spam,unbelievable new homes made easy im wanting to...
3,spam,4 color printing special request additional in...
4,spam,do not have money get software cds from here s...


In [18]:
df['text_type'].value_counts()

text_type
ham     14337
spam     6011
Name: count, dtype: int64

### text preproc.

In [68]:
spec_c = [".", ",", "?", "!"]
for c in spec_c:
    df['text'] = df['text'].replace(c, " " + c)

In [69]:
le = LabelEncoder()
df['text_type'] = le.fit_transform(df['text_type'])

In [70]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['text_type'], test_size=0.2, random_state=42)

In [71]:
tokenizer = Tokenizer(filters='"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(list(X_train))

sequences = tokenizer.texts_to_sequences(X_train)

print(f'''word categories count: {len(sequences[0])}
unique words count: {len(tokenizer.word_index)}
first line's window size: {len(tokenizer.texts_to_sequences([list(X_train)[0]])[0])}''')

word categories count: 13
unique words count: 57636
first line's window size: 13


In [72]:
max_sequence_length = max(len(seq) for seq in sequences)
print(max_sequence_length)

196


In [73]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_sequence_length)

In [88]:
from keras import Sequential
from keras.layers import SimpleRNN, Dense, GRU, LSTM, Conv1D, Dropout, Embedding, GlobalAveragePooling1D

In [106]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 64, input_length=max_sequence_length))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(GRU(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy','precision','recall'])

model.summary()

/Users/banoczymartin/OE projects II/Deep-Learning-Experiments/vince_dl/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_34 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_8      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [107]:
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=60, validation_data=(X_test_pad, y_test))

Epoch 1/10
272/272 ━━━━━━━━━━━━━━━━━━━━ 122s 437ms/step - accuracy: 0.6908 - loss: 0.6143 - precision: 0.3263 - recall: 0.0307 - val_accuracy: 0.8128 - val_loss: 0.3691 - val_precision: 0.9284 - val_recall: 0.3699
Epoch 2/10
272/272 ━━━━━━━━━━━━━━━━━━━━ 106s 391ms/step - accuracy: 0.8955 - loss: 0.2762 - precision: 0.9057 - recall: 0.7389 - val_accuracy: 0.9305 - val_loss: 0.1860 - val_precision: 0.9477 - val_recall: 0.7995
Epoch 3/10
272/272 ━━━━━━━━━━━━━━━━━━━━ 112s 411ms/step - accuracy: 0.9582 - loss: 0.1189 - precision: 0.9608 - recall: 0.8968 - val_accuracy: 0.9553 - val_loss: 0.1278 - val_precision: 0.9577 - val_recall: 0.8816
Epoch 4/10
272/272 ━━━━━━━━━━━━━━━━━━━━ 100s 366ms/step - accuracy: 0.9677 - loss: 0.1333 - precision: 0.9701 - recall: 0.9216 - val_accuracy: 0.9415 - val_loss: 0.1624 - val_precision: 0.9637 - val_recall: 0.8254
Epoch 5/10
272/272 ━━━━━━━━━━━━━━━━━━━━ 97s 357ms/step - accuracy: 0.9772 - loss: 0.0682 - precision: 0.9826 - recall: 0.9387 - val_accuracy: 0.

In [ ]:

# "text" column contains the message text and "text_type" contains the labels (spam or not)
X = df['text']
y = df['text_type']

# Convert labels to numerical values
le = LabelEncoder()
y = le.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to make them uniform length
max_sequence_length = 150  # Choose the maximum length of sequences
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Create model

In [ ]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_sequence_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

In [ ]:
#display the model summary
model.summary()

## Save & Load the model

In [ ]:
#Save the model
model.save("output/spam_or_ham_model.keras")

In [ ]:
#Load the model
model = tf.keras.models.load_model("output/spam_or_ham_model.keras")

# Evaluate the model

Evaluation of the model should be done on the test set, which is not available to us. Therefore, we will split the training data into training and validation sets. We will use the validation set to tune the hyperparameters of the model.

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Further evaluation metrics
y_pred = model.predict(X_test_pad)
y_pred = (y_pred > 0.5) # Convert the probabilities to binary predictions

In [ ]:
#visualize the loss and accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

#visualize the classification report
cm = confusion_matrix(y_test, y_pred)
plt.matshow(cm, cmap=plt.cm.Blues)
plt.colorbar()
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
#ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# average precision score
average_precision = average_precision_score(y_test, y_pred)
print("Average Precision Score:", average_precision)

# Precision-Recall curve 
# Precision: the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives
# Recall: the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives

precision, recall, thresholds = precision_recall_curve(y_test, y_pred) # Compute precision-recall pairs for different probability thresholds
plt.figure()
plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='c')
plt.xlabel('Recall') 
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall curve')
plt.show()

## Evaluate with user defined test sets

In [ ]:
my_text="Here is an easy way to make a lot of money quick. Click here to learn more."

In [ ]:
# Example of classifying a new text_message
#new_text_message = ["Your new text_message text goes here"]
new_text_message = [my_text]
new_text_message_seq = tokenizer.texts_to_sequences(new_text_message)
new_text_message_pad = pad_sequences(new_text_message_seq, maxlen=max_sequence_length)
prediction = model.predict(new_text_message_pad)


print("The text message is: ", new_text_message)
#Print the prediction (0 = ham, 1 = spam) and print the accuracy of the model
if prediction[0][0] >= 0.5:
    print("The text message is spam.")
    #print accuracy of the model
    print("At accuracy: ", accuracy)
    #print how sure it is that the text message is spam
    print("the model is ", prediction[0][0]*100, "% sure that the text message is spam.")
    
else:
    print("The text message is ham.")
    #print accuracy of the model
    print("At accuracy: ", accuracy)
    #print how sure it is that the text message is ham
    print("the model is ", (1-prediction[0][0])*100, "% sure that the text message is not spam (ham).")

# Data analysis and visualization

In [ ]:
# show percentage of spam and ham in the dataset
print("Percentage of spam in the dataset: ", (sum(y)/len(y))*100, "%")
print("Percentage of ham in the dataset: ", ((len(y)-sum(y))/len(y))*100, "%")

# show percentage of spam and ham in the test set
print("Percentage of spam in the test set: ", (sum(y_test)/len(y_test))*100, "%")
print("Percentage of ham in the test set: ", ((len(y_test)-sum(y_test))/len(y_test))*100, "%")

# show percentage of spam and ham in the training set
print("Percentage of spam in the training set: ", (sum(y_train)/len(y_train))*100, "%")
print("Percentage of ham in the training set: ", ((len(y_train)-sum(y_train))/len(y_train))*100, "%")

In [ ]:
# show a graph of the percentage of spam and ham in the dataset
objects = ('Spam', 'Ham')
y_pos = np.arange(len(objects))
performance = [sum(y), len(y)-sum(y)]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Number of messages')
plt.title('Spam vs Ham in the dataset')
plt.show()

In [ ]:
# visualize the length of the messages in the dataset
df['length'] = df['text'].apply(len)
df['length'].plot(bins=50, kind='hist')
plt.show()

#visualize the length of spam and ham messages in the dataset
df.hist(column='length', by='text_type', bins=50,figsize=(12,4))
plt.show()